# This notebook runs correctly
### But the query is inefficient

In [1]:
import pandas as pd
# I can give a number or use None to remove maximum ceiling & display all columns
pd.options.display.max_columns = None


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [3]:
import random

# Setting up GPT 

In [5]:
from langchain.chat_models import ChatOpenAI

# Use temperature=0 to get the same results every time
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key = openai_api_key)


# Following WaPo example

## Create training data in excel
I filtered the data by those mentioning emails/ correspondence/ text messages and mostly trained on that but also included some empty ones

In [5]:
# Read Document
df_public_records_for_emails=pd.read_csv("save_df_public_records_25102023.csv")
df_public_records_for_emails

,Unnamed: 0,pr_req_url,duplicate_pr_req_url,uploaded_dates,extracted_received_date,received_via,first_file_uploaded_date,contact,request_opened_date,request_opened_time,request_opened_by,last_file_uploaded_date,duration_from_received_to_first_uploaded_date,duration_from_received_to_last_uploaded_date,request_opened_datetime,duration_from_open_to_last_uploaded_date,status_scraped,full_request_message,is_responder_requesting_email,request_closed_content_text,exploded_dep_names,num_of_departments,original_dep_list,exploded_dep_renamed,req_id,req_url,req_closed_time,duration_from_received_to_closed,duration_from_received_to_closed_among_closed_requests,exploded_dep_renamed_saving,frequency_of_closure_text,extended_category_request_closed_content_text,category_request_closed_content_text
0,0,https://miami.nextrequest.com/requests/18-1,NaN,[],2018-01-01,web,NaN,Ryan Steed,2018-01-01,7:15 AM,the requester,NaN,NaN,NaN,2018-01-01 7:15:00,NaN,Closed,This should be directed to:\nRyan Steed\nThe f...,yes,Previous Request was reopened. This Request wa...,No departments assigned,1,['No departments assigned'],No departments assigned,18-1,https://miami.nextrequest.com/client/requests/...,2018-01-02 10:13:00,1 days 10:13:00,1 days 10:13:00,No departments assigned,1.0,duplicate,referred_or_fowarded_or_duplicate
1,1,https://miami.nextrequest.com/requests/18-10,First Duplicate,[],2018-01-02,web,NaN,Ryan Steed,2018-01-02,3:28 PM,the requester,NaN,NaN,NaN,2018-01-02 15:28:00,NaN,Closed,We are requesting all police reports for any i...,no,Request was forwarded to Police,No departments assigned,1,['No departments assigned'],No departments assigned,18-10,https://miami.nextrequest.com/client/requests/...,2018-01-03 9:46:00,1 days 09:46:00,1 days 09:46:00,No departments assigned,4.0,referred_to_police,referred_or_fowarded_or_duplicate
2,2,https://miami.nextrequest.com/requests/18-100,First Duplicate,[],2018-01-24,email,NaN,Michelle Redruello,2018-01-24,10:00 AM,Staff,NaN,NaN,NaN,2018-01-24 10:00:00,NaN,Closed,"Dear Chief Jorge: \n \nAs you know, this firm ...",yes,FulfilledAll responsive and non-exempt public ...,No departments assigned,1,['No departments assigned'],No departments assigned,18-100,https://miami.nextrequest.com/client/requests/...,2018-01-24 10:01:00,0 days 10:01:00,0 days 10:01:00,No departments assigned,376.0,documents_provided_download,fulfilled
3,3,https://miami.nextrequest.com/requests/18-1000,First Duplicate,[],2018-06-28,email,NaN,Stephanie Schloss-Sassi,2018-06-28,11:02 AM,Staff,NaN,NaN,NaN,2018-06-28 11:02:00,NaN,Closed,I would also like to request any email that Ad...,yes,Closed at Direction/Approval of the RequestorT...,I.T.,1,['I.T.'],I.T.,18-1000,https://miami.nextrequest.com/client/requests/...,2018-06-28 11:03:00,0 days 11:03:00,0 days 11:03:00,I.T.,549.0,Closed_at_Direction_or_Approval_of_the_Requestor,closed_at_approval_of_requestor
4,4,https://miami.nextrequest.com/requests/18-1001,NaN,['Uploaded: 12/18/2018'],2018-06-28,email,2018-12-18,Ryan Steed,2018-06-28,11:03 AM,Staff,2018-12-18,173 days,173 days,2018-06-28 11:03:00,172 days 12:57:00,Closed,"Please, I would like to make a public records ...",no,The City does not have any responsive document...,Building,1,['Building'],Building,18-1001,https://miami.nextrequest.com/client/requests/...,2018-12-18 9:28:00,173 days 09:28:00,173 days 09:28:00,Building,2854.0,no_responsive_documents_available,no_responsive_documents_available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24533,24533,https://miami.nextrequest.com/requests/23-995,NaN,[],2023-03-16,web,NaN,Michael Mesa,2023-03-16,2:39 PM,the requester,NaN,NaN,NaN,2023-03-16 14:39:00,NaN,Closed,Property: Core Link At Douglas Address:3060 SW...,NaN,The City does not have any responsive document...,Fire - Property Search,1,['Fire - Property Search'],Fire - Property Search,23-995,https://miami.nextrequest.com/client/requests/...,2023-03-17 

# Filter by cases where training data is filled

In [6]:
known_df_public_records_for_emails = df_public_records_for_emails[df_public_records_for_emails.is_responder_requesting_email.notna()].copy()
unknown_df_public_records_for_emails = df_public_records_for_emails[df_public_records_for_emails.is_responder_requesting_email.isna()].copy()

In [7]:
known_df_public_records_for_emails.shape


(487, 33)

# Fill Missing Values

In [8]:
known_df_public_records_for_emails['full_request_message'].fillna('filling missing_values for training model to function', inplace=True)
unknown_df_public_records_for_emails['full_request_message'].fillna('filling missing_values for training model to function', inplace=True)


# Now GPT
### Prep data

In [9]:
df_public_records_for_emails_head=df_public_records_for_emails[0:10]
df_public_records_for_emails_head

,Unnamed: 0,pr_req_url,duplicate_pr_req_url,uploaded_dates,extracted_received_date,received_via,first_file_uploaded_date,contact,request_opened_date,request_opened_time,request_opened_by,last_file_uploaded_date,duration_from_received_to_first_uploaded_date,duration_from_received_to_last_uploaded_date,request_opened_datetime,duration_from_open_to_last_uploaded_date,status_scraped,full_request_message,is_responder_requesting_email,request_closed_content_text,exploded_dep_names,num_of_departments,original_dep_list,exploded_dep_renamed,req_id,req_url,req_closed_time,duration_from_received_to_closed,duration_from_received_to_closed_among_closed_requests,exploded_dep_renamed_saving,frequency_of_closure_text,extended_category_request_closed_content_text,category_request_closed_content_text
0,0,https://miami.nextrequest.com/requests/18-1,NaN,[],2018-01-01,web,NaN,Ryan Steed,2018-01-01,7:15 AM,the requester,NaN,NaN,NaN,2018-01-01 7:15:00,NaN,Closed,This should be directed to:\nRyan Steed\nThe f...,yes,Previous Request was reopened. This Request wa...,No departments assigned,1,['No departments assigned'],No departments assigned,18-1,https://miami.nextrequest.com/client/requests/...,2018-01-02 10:13:00,1 days 10:13:00,1 days 10:13:00,No departments assigned,1.0,duplicate,referred_or_fowarded_or_duplicate
1,1,https://miami.nextrequest.com/requests/18-10,First Duplicate,[],2018-01-02,web,NaN,Ryan Steed,2018-01-02,3:28 PM,the requester,NaN,NaN,NaN,2018-01-02 15:28:00,NaN,Closed,We are requesting all police reports for any i...,no,Request was forwarded to Police,No departments assigned,1,['No departments assigned'],No departments assigned,18-10,https://miami.nextrequest.com/client/requests/...,2018-01-03 9:46:00,1 days 09:46:00,1 days 09:46:00,No departments assigned,4.0,referred_to_police,referred_or_fowarded_or_duplicate
2,2,https://miami.nextrequest.com/requests/18-100,First Duplicate,[],2018-01-24,email,NaN,Michelle Redruello,2018-01-24,10:00 AM,Staff,NaN,NaN,NaN,2018-01-24 10:00:00,NaN,Closed,"Dear Chief Jorge: \n \nAs you know, this firm ...",yes,FulfilledAll responsive and non-exempt public ...,No departments assigned,1,['No departments assigned'],No departments assigned,18-100,https://miami.nextrequest.com/client/requests/...,2018-01-24 10:01:00,0 days 10:01:00,0 days 10:01:00,No departments assigned,376.0,documents_provided_download,fulfilled
3,3,https://miami.nextrequest.com/requests/18-1000,First Duplicate,[],2018-06-28,email,NaN,Stephanie Schloss-Sassi,2018-06-28,11:02 AM,Staff,NaN,NaN,NaN,2018-06-28 11:02:00,NaN,Closed,I would also like to request any email that Ad...,yes,Closed at Direction/Approval of the RequestorT...,I.T.,1,['I.T.'],I.T.,18-1000,https://miami.nextrequest.com/client/requests/...,2018-06-28 11:03:00,0 days 11:03:00,0 days 11:03:00,I.T.,549.0,Closed_at_Direction_or_Approval_of_the_Requestor,closed_at_approval_of_requestor
4,4,https://miami.nextrequest.com/requests/18-1001,NaN,['Uploaded: 12/18/2018'],2018-06-28,email,2018-12-18,Ryan Steed,2018-06-28,11:03 AM,Staff,2018-12-18,173 days,173 days,2018-06-28 11:03:00,172 days 12:57:00,Closed,"Please, I would like to make a public records ...",no,The City does not have any responsive document...,Building,1,['Building'],Building,18-1001,https://miami.nextrequest.com/client/requests/...,2018-12-18 9:28:00,173 days 09:28:00,173 days 09:28:00,Building,2854.0,no_responsive_documents_available,no_responsive_documents_available
5,5,https://miami.nextrequest.com/requests/18-1002,NaN,['Uploaded: 06/28/2018'],2018-06-28,web,2018-06-28,Ryan Steed,2018-06-28,2:06 PM,the requester,2018-06-28,0 days,0 days,2018-06-28 14:06:00,-1 days +09:54:00,Closed,Bid Results for:\n- Plumbing Maintenance and R...,no,All documents believed to be responsive have b...,Procurement,1,['Procurement'],Procurement,18-1002,https://miami.nextrequest.com/client/requests/...,2018-06-28 15:50:00,0 days 15:50:00,0 days 15:50:00,Procurement,4977.0,documents_via_custodian,fulfilled
6,6,https://miami.nextreques

#### Make a list of examples of yes and no answers

In [11]:
yes_known_df_public_records_for_emails=known_df_public_records_for_emails[known_df_public_records_for_emails['is_responder_requesting_email']=="yes"]
no_known_df_public_records_for_emails=known_df_public_records_for_emails[known_df_public_records_for_emails['is_responder_requesting_email']=="no"]

In [12]:
yes_known_df_public_records_for_emails_list=yes_known_df_public_records_for_emails['full_request_message'].tolist()
no_known_df_public_records_for_emails_list=no_known_df_public_records_for_emails['full_request_message'].tolist()

# Ask GPT to read

In [33]:
# Import necessary libraries
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter


# Assume df_public_records_for_emails is your dataframe and llm represents a language model like ChatGPT

# Create embeddings
embeddings = HuggingFaceEmbeddings(model_name='multi-qa-mpnet-base-dot-v1')

# Instead of loading PDF, we will work directly with the dataframe column
data = df_public_records_for_emails_head['full_request_message'].tolist()

# Define the SimpleDocument class with a 'page_content' attribute
class SimpleDocument:
    def __init__(self, text, metadata=None):
        self.page_content = text
        self.metadata = metadata if metadata is not None else {}


# Convert list of strings to list of Document objects
docs = [SimpleDocument(text) for text in data]

# Create a Chroma vector store from the documents
db = Chroma.from_documents(docs, embeddings)


# Initialize the RetrievalQA chain with the correct type
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Replace with the correct chain type for your application
    retriever=db.as_retriever()
)

# Iterate over the dataframe and check each message
for index, text in enumerate(data):
    query = f"""This is a request for public records. 
            Sometimes, requests may include a request for records, documents, transcripts etc. only but not a request for communication, correspondence, emails or text messages. This would mean this is not a correspondence-related request.
            However, sometimes a request includes specifically a request for communication(s), correspondence, email(s) or text messages. This means this is a correspondence-related request.
            I have looked at some of the requests myself. I see that the non-correspondence related requests are strictly for records, documents, transcripts, permits among others as public records are diverse but the non-correspondence related requests do not include requests for communication, correspondence, emails or text messages
            Please note that if the requester is asking for the request results to be emailed to them or leaves their own email address, that is not necessarily a request for communication, correspondence, emails or text messages but it might be if they are additionally also requesting communication, correspondence, emails or text messages
            What is a correspondence-related request is a request where the person wants to be provided with the communication, correspondence, emails or text messages
            Here is an example of times when the answer is Yes: {random.sample(yes_known_df_public_records_for_emails_list, 1)}
            Here is an example of times when the answer is No: {random.sample(no_known_df_public_records_for_emails_list, 1)}
            With all that in mind, is this request a correspondence-related request, that is,  does it include a specific request for communication, correspondence, emails or text messages?"""
    question = query
    answer = qa.run(question)
    # Process the answer to determine Yes or No
    df_public_records_for_emails_head.at[index, 'request_for_emails'] = 'Yes' if 'yes' in answer.lower() else 'No'

# Now df_public_records_for_emails should have a new column 'request_for_emails' with 'Yes' or 'No' values


Using embedded DuckDB without persistence: data will be transient


In [34]:
df_public_records_for_emails_head

,Unnamed: 0,pr_req_url,duplicate_pr_req_url,uploaded_dates,extracted_received_date,received_via,first_file_uploaded_date,contact,request_opened_date,request_opened_time,request_opened_by,last_file_uploaded_date,duration_from_received_to_first_uploaded_date,duration_from_received_to_last_uploaded_date,request_opened_datetime,duration_from_open_to_last_uploaded_date,status_scraped,full_request_message,is_responder_requesting_email,request_closed_content_text,exploded_dep_names,num_of_departments,original_dep_list,exploded_dep_renamed,req_id,req_url,req_closed_time,duration_from_received_to_closed,duration_from_received_to_closed_among_closed_requests,exploded_dep_renamed_saving,frequency_of_closure_text,extended_category_request_closed_content_text,category_request_closed_content_text,request_for_emails
0,0,https://miami.nextrequest.com/requests/18-1,NaN,[],2018-01-01,web,NaN,Ryan Steed,2018-01-01,7:15 AM,the requester,NaN,NaN,NaN,2018-01-01 7:15:00,NaN,Closed,This should be directed to:\nRyan Steed\nThe f...,yes,Previous Request was reopened. This Request wa...,No departments assigned,1,['No departments assigned'],No departments assigned,18-1,https://miami.nextrequest.com/client/requests/...,2018-01-02 10:13:00,1 days 10:13:00,1 days 10:13:00,No departments assigned,1.0,duplicate,referred_or_fowarded_or_duplicate,Yes
1,1,https://miami.nextrequest.com/requests/18-10,First Duplicate,[],2018-01-02,web,NaN,Ryan Steed,2018-01-02,3:28 PM,the requester,NaN,NaN,NaN,2018-01-02 15:28:00,NaN,Closed,We are requesting all police reports for any i...,no,Request was forwarded to Police,No departments assigned,1,['No departments assigned'],No departments assigned,18-10,https://miami.nextrequest.com/client/requests/...,2018-01-03 9:46:00,1 days 09:46:00,1 days 09:46:00,No departments assigned,4.0,referred_to_police,referred_or_fowarded_or_duplicate,Yes
2,2,https://miami.nextrequest.com/requests/18-100,First Duplicate,[],2018-01-24,email,NaN,Michelle Redruello,2018-01-24,10:00 AM,Staff,NaN,NaN,NaN,2018-01-24 10:00:00,NaN,Closed,"Dear Chief Jorge: \n \nAs you know, this firm ...",yes,FulfilledAll responsive and non-exempt public ...,No departments assigned,1,['No departments assigned'],No departments assigned,18-100,https://miami.nextrequest.com/client/requests/...,2018-01-24 10:01:00,0 days 10:01:00,0 days 10:01:00,No departments assigned,376.0,documents_provided_download,fulfilled,No
3,3,https://miami.nextrequest.com/requests/18-1000,First Duplicate,[],2018-06-28,email,NaN,Stephanie Schloss-Sassi,2018-06-28,11:02 AM,Staff,NaN,NaN,NaN,2018-06-28 11:02:00,NaN,Closed,I would also like to request any email that Ad...,yes,Closed at Direction/Approval of the RequestorT...,I.T.,1,['I.T.'],I.T.,18-1000,https://miami.nextrequest.com/client/requests/...,2018-06-28 11:03:00,0 days 11:03:00,0 days 11:03:00,I.T.,549.0,Closed_at_Direction_or_Approval_of_the_Requestor,closed_at_approval_of_requestor,Yes
4,4,https://miami.nextrequest.com/requests/18-1001,NaN,['Uploaded: 12/18/2018'],2018-06-28,email,2018-12-18,Ryan Steed,2018-06-28,11:03 AM,Staff,2018-12-18,173 days,173 days,2018-06-28 11:03:00,172 days 12:57:00,Closed,"Please, I would like to make a public records ...",no,The City does not have any responsive document...,Building,1,['Building'],Building,18-1001,https://miami.nextrequest.com/client/requests/...,2018-12-18 9:28:00,173 days 09:28:00,173 days 09:28:00,Building,2854.0,no_responsive_documents_available,no_responsive_documents_available,Yes
5,5,https://miami.nextrequest.com/requests/18-1002,NaN,['Uploaded: 06/28/2018'],2018-06-28,web,2018-06-28,Ryan Steed,2018-06-28,2:06 PM,the requester,2018-06-28,0 days,0 days,2018-06-28 14:06:00,-1 days +09:54:00,Closed,Bid Results for:\n- Plumbing Maintenance and R...,no,All documents believed to be responsive have b...,Procurement,1,['Procurement'],Procurement,18-1002,https://miami.nextrequest.com/client/requests/...,2018-06-28 15:50:00,0 days 15:50:00,0 days 15:50:00,Procurement,4977.0,documents_via_custodian,

In [77]:
df_public_records_for_emails_head.to_csv("df_public_records_for_emails_head.csv")